# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **Mont-Tremblant, Quebec, Canada**, or **Canada** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Mont-Tremblant, Quebec, Canada** to Ann Arbor, USA. In that case at least one source file must be about **Mont-Tremblant, Quebec, Canada**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Mont-Tremblant, Quebec, Canada** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

**Dataset 1**
- Ironman 2018 results in Mont-Tremblant per categories
- https://www.endurance-data.com/en/results/270-ironman-mont-tremblant/all/
    
**Dataset 2**
- Distance from Canada to other countries
- https://www.distancefromto.net/distance-from-canada-country
    
**Dataset 3**
- List of FIPS country codes
- https://en.wikipedia.org/wiki/List_of_FIPS_country_codes

**Dataset 4**
- List of ISO 3166-1 alpha-3 country codes
- https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
    
**TODO**
- clean ++ general describe on datasets
- piecharts avg nationalities (total)
- scatter plot everything vs categories (should see clusters especially for time results)
- % of country represented
- join df_distance && df_ironman
- general correlation plot every column vs. distance from Canada (international athletes perform better, per category)
- Do international atheletes that travel further from their home country perform better than local atheletes? 
- markdown cell explaining datasets & hypothesis formulation & p-value
    -- did not use more recent weather data because required lots of cleaning and parsing (NOAA, NCEI)
- groupby category, get mean + std + preview distributions
- compute p-value and reject/fail-to-reject hypothesis
- markdown cell explaining results and visualizations & limitations
    = Limitations
        -> calculating distance from country of origin, not town where athele lives, 
            even if still lives in home country, not using hometown (very rough estimate)



## Load & clean data

In [8]:
%%javascript
// Disable scrolling
//    https://stackoverflow.com/questions/41641205/how-to-avoid-output-into-scrollable-frames-in-jupyter-notebook
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib notebook
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [162]:
class Dataloader():
    def __init__(self):
        self.countries = self._load_countries()
        self.ironman = self._load_ironman()
        self.distances = self._load_dists()

    def _load_countries(self):
        df = pd.merge(self._load_alpha3(), self._load_fips(), on='Name')
        return df

    def _fips_to_alpha(self, code):
        return self.countries['Alpha'][self.countries['FIPS'] == code].values[0]

    def _alpha_to_fips(self, code):
        try:
            return self.countries['FIPS'][self.countries['Alpha'] == code
                                         ].values[0]
        except Exception as e:
            # TODO print error and stop program
            pass

    def _load_alpha3(self, fname='data/alpha3.csv'):
        df = pd.read_csv(fname, names=['Alpha', 'Name'])
        return df

    def _load_fips(self, fname='data/FIPS.csv'):
        df = pd.read_csv(fname, names=['FIPS', 'Name'])
        return df

    def _load_ironman(self, fname='data/ironman2018.csv'):
        # TODO convert alpha3 -> FIPS
        # - strip whitespace
        df = pd.read_csv(
            fname,
            names=[
                'Overall', 'GenderRank', 'DivRank', 'Name', 'BIB', 'AgeGroup',
                'Country', 'Swim', 'Bike', 'Run', 'Total'
            ],
            dtype={
                'Overall': np.int64,
            },
            converters=dict(
                {'Country': lambda x: self._alpha_to_fips(x.strip())}, **{
                    k: lambda x: pd.to_datetime(x, format="%H:%M:%S").time()
                    for k in ['Swim', 'Bike', 'Run']
                }
            )
        )
        return df

    def _load_dists(self, fname='data/distances.csv'):
        df = pd.read_csv(fname)
        return df


dl = Dataloader()
dl.ironman.head()

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given

<IPython.core.display.Javascript object>